# Verisk - computer purchase problem

*by Jolanta Śliwa*

## Problem Statement
Suppose you're trying to help a company determine which computers to purchase.
### Data - utilization data by employee:
The company has been able to pull utilization data by employee that classifies users into 3 bins, depending on how much they use their computer in their work:
* Low usage - spends a lot of time in meetings, checking email, doing people management
* Average usage - requires some compute power, with balanced mix of heads down/technical work along with a
good amount of meetings/email writing
* High usage - power user, relies heavily on computer performance


In [1]:
import pandas as pd


utilization = pd.read_csv(
    "https://raw.githubusercontent.com/shubhamkalra27/dsep-2020/main/datasets/util_b_emp.csv"
)

In [2]:
utilization.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   employee_id      146 non-null    int64 
 1   utilization_bin  146 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.4+ KB


In [3]:
utilization.head()

,employee_id,utilization_bin
0,1743,high
1,1752,high
2,1758,high
3,1825,high
4,1842,high


Checking types of utilization

In [4]:
utilization["utilization_bin"].unique()

array(['high', 'medium', 'low'], dtype=object)

average usage is stored as medium

## Data - survey
Additionally, they've surveyed employees to collect the relative importance of the following variables describing a
computer's performance:
* Memory
* Processing
* Storage
* Price inverse - this metric was given to you by the company as you can see in the dataset, with the directive that
price inverse being fixed at a 25% weight in the purchase decision

In [5]:
survey = pd.read_csv(
    "https://raw.githubusercontent.com/shubhamkalra27/dsep-2020/main/datasets/survey_emp.csv"
)

In [6]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   employee_id    146 non-null    int64  
 1   memory         146 non-null    float64
 2   processing     146 non-null    float64
 3   storage        146 non-null    float64
 4   inverse_price  146 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 5.8 KB


In [7]:
survey.head()

,employee_id,memory,processing,storage,inverse_price
0,1743,0.375,0.225,0.150,0.25
1,1752,0.450,0.225,0.075,0.25
2,1758,0.375,0.300,0.075,0.25
3,1825,0.300,0.300,0.150,0.25
4,1842,0.300,0.300,0.150,0.25


Checking whether we have 100% in every column

In [8]:
for i, row in survey.iterrows():
    if sum(row[1:]) != 1:
        print("problem")
        break
print("ok")

ok


## Data - computers
Lastly, the company is looking to purchase a maximum of 3 different computer models, and have compiled the following
list scoring their memory, processing, storage, and relative price. Each dimension is scored from 0-10, with 10 being the best.

In [9]:
computers = pd.read_csv(
    "https://raw.githubusercontent.com/shubhamkalra27/dsep-2020/main/datasets/vendor_options.csv"
)

In [10]:
computers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   computer_id    11 non-null     int64  
 1   memory         11 non-null     int64  
 2   processing     11 non-null     int64  
 3   storage        11 non-null     int64  
 4   inverse_price  11 non-null     float64
dtypes: float64(1), int64(4)
memory usage: 568.0 bytes


In [11]:
computers.head()

,computer_id,memory,processing,storage,inverse_price
0,13,5,7,10,2.7
1,16,9,8,9,1.3
2,4,8,9,10,1.0
3,1,8,8,9,1.7
4,3,5,4,4,5.7


Checking "real" range of scores:

In [12]:
print(computers.max())

computer_id      20.0
memory            9.0
processing       10.0
storage          10.0
inverse_price     5.7
dtype: float64


In [13]:
print(computers.min())

computer_id      1.0
memory           5.0
processing       4.0
storage          4.0
inverse_price    1.0
dtype: float64


## Task
**Given this information, provide the company with a recommendation on which computers to purchase.**

List of parameters:

In [14]:
parameters = computers.columns[1:]
print(parameters)

Index(['memory', 'processing', 'storage', 'inverse_price'], dtype='object')


It will be more convenient for me to store all employees related data in one DataFrame instead of two.

Merging survey and utilization into employees column:

In [15]:
employees = utilization.merge(survey, left_on="employee_id", right_on="employee_id")

In [16]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146 entries, 0 to 145
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   employee_id      146 non-null    int64  
 1   utilization_bin  146 non-null    object 
 2   memory           146 non-null    float64
 3   processing       146 non-null    float64
 4   storage          146 non-null    float64
 5   inverse_price    146 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 8.0+ KB


In [17]:
employees.head()

,employee_id,utilization_bin,memory,processing,storage,inverse_price
0,1743,high,0.375,0.225,0.150,0.25
1,1752,high,0.450,0.225,0.075,0.25
2,1758,high,0.375,0.300,0.075,0.25
3,1825,high,0.300,0.300,0.150,0.25
4,1842,high,0.300,0.300,0.150,0.25


Normally (if we would like to make some predictions) it would be better to store "object" (nominal data - categorical data) using one-hot-encoding but in this case it is more convenient for me to leave it like that.

Under we will see outputs of different metrics. For each there will be solution provided by:
* Simulated Annealing
* Naive algorithm

Results given by algorithms: id of computer in DataFrame - not the one in colum computer_id

In [18]:
from problem import (
    ProblemNothing,
    ProblemMax,
    ProblemScale,
    ProblemMaxHalf,
    ProblemScaleHalf,
)

* ProblemNothing - metric without using utilization info
* ProblemMax - metric where every group have a different max score (3, 7, 10)
* ProblemMaxHalf - like above with different values (5, 7.5, 10)
* ProblemScale - metric where we scale computers scores by multiplying them (3/1, 3/2, 3/3)
* ProblemScaleHalf - like above with different values (4/2, 4/3, 4/4)

In [19]:
from simulated_annealing import SimulatedAnnealing, SimulatedAnnealingConfig
from naive_solution import Naive

### No utilization value

In [20]:
prob_n = ProblemNothing(computers, employees)

In [21]:
annealing_n = SimulatedAnnealing(SimulatedAnnealingConfig(), prob_n)

annealing_n.solve()

SOLUTION:
 Best: [10, 1, 8]


[10, 1, 8]

In [23]:
naive_n = Naive(prob_n)

naive_n.solve()

SOLUTION:
 Best: [1, 2, 8]


[1, 2, 8]

As we can see simulated annealing sometimes returns solutions that aren't optimal
Let's try one more time

In [26]:
annealing_n = SimulatedAnnealing(SimulatedAnnealingConfig(), prob_n)

annealing_n.solve()

SOLUTION:
 Best: [8, 2, 1]


[8, 2, 1]

This time we manage to get "the best" solution for this metric

Let's see how it looks like:

In [28]:
prob_n.calculate_state_cost([10, 1, 8])

1024.650000000001

In [29]:
prob_n.calculate_state_cost([8, 2, 1])

1024.650000000001

So actually there are at least 2 optimal solutions

### Problem MAX
#### max: (3, 7, 10)

In [20]:
prob_max = ProblemMax(computers, employees)

In [30]:
annealing_max = SimulatedAnnealing(SimulatedAnnealingConfig(), prob_max)

result_max = annealing_max.solve()

SOLUTION:
 Best: [4, 5, 1]


In [31]:
top_max = prob_max.get_wanted_computers(result_max)

In [32]:
print(top_max.keys())

dict_keys([1, 5, 4])


In [24]:
naive_max = Naive(prob_max)

result_max_n = naive_max.solve()

SOLUTION:
 Best: [1, 4, 5]


In [26]:
print(prob_max.get_wanted_computers(result_max_n).keys())

dict_keys([1, 5, 4])


#### max: (5, 7.5, 10)

In [41]:
prob_max_half = ProblemMaxHalf(computers, employees)

In [42]:
annealing_max_half = SimulatedAnnealing(SimulatedAnnealingConfig(), prob_max_half)

result_max_half = annealing_max_half.solve()

SOLUTION:
 Best: [1, 3, 4]


In [43]:
print(prob_max_half.get_wanted_computers(result_max_half).keys())

dict_keys([1, 3, 4])


In [44]:
naive_max_half = Naive(prob_max_half)

result_max_half_n = naive_max_half.solve()

SOLUTION:
 Best: [1, 3, 4]


In [45]:
print(prob_max_half.get_wanted_computers(result_max_half_n).keys())

dict_keys([1, 3, 4])


### Problem Scale
#### scale: (3/1, 3/2, 3/3)

In [46]:
prob_scale = ProblemScale(computers, employees)

In [47]:
annealing_scale = SimulatedAnnealing(SimulatedAnnealingConfig(), prob_scale)

result_scale = annealing_scale.solve()

SOLUTION:
 Best: [4, 5, 1]


In [49]:
print(prob_scale.get_wanted_computers(result_scale).keys())

dict_keys([1, 5, 4])


In [48]:
naive_scale = Naive(prob_scale)

result_scale_n = naive_scale.solve()

SOLUTION:
 Best: [1, 4, 5]


In [50]:
print(prob_scale.get_wanted_computers(result_scale_n).keys())

dict_keys([1, 5, 4])


#### scale: (4/2, 4/3, 4/4)

In [52]:
prob_scale_half = ProblemScaleHalf(computers, employees)

In [53]:
annealing_scale_half = SimulatedAnnealing(SimulatedAnnealingConfig(), prob_scale_half)

result_scale_half = annealing_scale_half.solve()

SOLUTION:
 Best: [5, 1, 3]


In [54]:
print(prob_scale_half.get_wanted_computers(result_scale_half).keys())

dict_keys([1, 3, 5])


In [57]:
naive_scale_half = Naive(prob_scale_half)

result_scale_half_n = naive_scale_half.solve()

SOLUTION:
 Best: [1, 3, 5]


In [58]:
print(prob_scale_half.get_wanted_computers(result_scale_half_n).keys())

dict_keys([1, 3, 5])


There is still a question which 3 computers are the best for that company?

We can see according to all presented metrics computer in row nr 1 is always in top three

Others can vary depending on metric we use. We can choose one of the above mathod but there are only 3 candidates for 2 positions.
nr five wasn't choosen only by problem max

In [59]:
print(result_max_half)

[1, 3, 4]


In [60]:
print(prob_max_half.calculate_state_cost(result_max_half))

885.2749999999984


In [61]:
print(prob_max_half.calculate_state_cost([1, 3, 5]))

884.3749999999992


In [62]:
print(prob_max_half.calculate_state_cost([1, 5, 4]))

876.6750000000017


For that metric there is not a big difference between computer 5 and 4
nr 3 seems to better option to leave

On the other hand we see that nr 4 also was't choosen only by on metric: scale half
So it seems that we have final three

In [63]:
print(result_scale_half)

[5, 1, 3]


In [66]:
print(prob_scale_half.calculate_state_cost(result_scale_half))

1120.6249999999986


In [65]:
print(prob_scale_half.calculate_state_cost([1, 5, 4]))

1112.025000000003


In [67]:
print(prob_scale_half.calculate_state_cost([1, 4, 3]))

1112.624999999997


Computer nr 4 seems to be less valuable and changing it with 3 or 5 give almost the same results

Just in case: Let's check nr 3 - wasn't choseen by 2 metrics

In [69]:
print(result_max)

[4, 5, 1]


In [70]:
print(prob_max.calculate_state_cost(result_max))

852.549999999999


In [71]:
print(prob_max.calculate_state_cost([3, 5, 1]))

840.5500000000009


In [72]:
print(prob_max.calculate_state_cost([3, 4, 1]))

834.1249999999976


In [73]:
print(result_scale)

[4, 5, 1]


In [77]:
print(prob_scale.calculate_state_cost(result_scale))

1151.0499999999988


In [74]:
print(prob_scale.calculate_state_cost([3, 5, 1]))

1139.0500000000004


In [75]:
print(prob_scale.calculate_state_cost([3, 4, 1]))

1132.624999999997


As we can see using above metrics and adding computer 3 instead of any other (4 or 5) results in a bigger i other cases derease of cost value

Under we I displayed top three computers:

In [78]:
computers.iloc[[1, 4, 5]]

,computer_id,memory,processing,storage,inverse_price
1,16,9,8,9,1.3
4,3,5,4,4,5.7
5,2,6,7,7,3.3


seems like we ended up with rather ballanced final state

In [80]:
computers.iloc[[1, 4, 5]]["computer_id"]

1    16
4     3
5     2
Name: computer_id, dtype: int64

And here we have computer 3 as an addiction

In [79]:
computers.iloc[[3]]

,computer_id,memory,processing,storage,inverse_price
3,1,8,8,9,1.7


it is simmilar to computer nr 3 but with a slightly better price but worse memory